In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/lorenzo/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/lorenzo/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/lorenzo/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import csv
import tweepy
import pandas as pd

import re
import string

# non riuscito a importare i seguenti package
#import language_check
#from pycontractions import Contractions
#import gensim.downloader as api

from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist, classify, NaiveBayesClassifier


#-------------------------- processing ------------------------------
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# hyper-parameters tuning
from sklearn.model_selection import GridSearchCV

# classifiers
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB,ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


In [ ]:
#eliminate emoticon
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
# Create variables for each key, secret, token
    consumer_key = "IG3sIHi0ovFvy7ZVygb928qzu"
    consumer_secret = "UXCsm2QYnLTWvliXTq9CPUPgY6oiDFPrWeOFY1u3x6Wv3SX0A5"
    access_token = "1301906875454304263-CQ8EKyDnuxymqez9EWtznKwQXhDZUH"
    access_token_secret = "Yln6i4Mv6PXAmZY8Vz22RGW86PeW6b1y4YZsIEyGDHwcM"

    # Set up OAuth and integrate with API
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    
    api = tweepy.API(auth)

In [ ]:
#public_tweets = api.home_timeline(q="Apple",lang="en")
#print(public_tweets[1].text)
#tweetTesting = public_tweets[0].text
tweetTesting = "\U0001F633 Apple 2021 @Apple “@Luigi “@Market update, we ’ got her back, so ’ ’ going to try and help her understand that she can go out, come back and be safe. https://t.co/Uyef7y0gxf"
print(tweetTesting)



😳 Apple 2021 @Apple “@Luigi “@Market update, we ’ got her back, so ’ ’ going to try and help her understand that she can go out, come back and be safe. https://t.co/Uyef7y0gxf


In [ ]:
#removing urls
tweetTesting = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                      '(?:%[0-9a-fA-F][0-9a-fA-F]))+','',  tweetTesting)
    
print(tweetTesting)


😳 Apple 2021 @Apple “@Luigi “@Market update, we ’ got her back, so ’ ’ going to try and help her understand that she can go out, come back and be safe. 


In [ ]:
#removing emoticon
tweetTesting = remove_emoji(tweetTesting)


In [ ]:
tweetTesting = re.sub("(@[A-Za-z0-9_]+)","", tweetTesting)
print(tweetTesting)

 Apple 2021  “ “ update, we ’ got her back, so ’ ’ going to try and help her understand that she can go out, come back and be safe. 


In [ ]:
tweetTesting = tweetTesting.lower()
print(tweetTesting)

 apple 2021  “ “ update, we ’ got her back, so ’ ’ going to try and help her understand that she can go out, come back and be safe. 


In [ ]:
#language-check non riuscito a installarlo
#contractions non riuscito a installarlo
#FAIL


In [ ]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
for punct_sign in string.punctuation:
    tweetTesting = tweetTesting.replace(punct_sign," ")
#others puntuations not within the set of string.punctuation->{!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~}
tweetTesting = tweetTesting.replace("’"," ")
tweetTesting = tweetTesting.replace("”"," ")
tweetTesting = tweetTesting.replace("“"," ")
tweetTesting = tweetTesting.replace("\n"," ")

print(tweetTesting)


    

 apple 2021      update  we   got her back  so     going to try and help her understand that she can go out  come back and be safe  


In [ ]:
# removing numbers  
tweetTesting = re.sub(r'\d+', '', tweetTesting)

In [ ]:
#controllo se ci sono 10 positivi, 10 negativi e altrettanti neutrali

#lettura file csv e prendo un tweet

#eliminazione di link(URL,href)

#eliminazione di emoticon

#(mybe) correzione slang americani

#eliminazione di . , : ; - _ ! ) ( ? !

#salvataggio tweet pulito

#lemmazation

#eliminazione dei stop word

#dizionario inglese e personalizzato{love,ugly,doesn't,work,wow,give,good...}

#calcolare TF-IDF

#BOW

#pipe1: BOW+SVM_dict_base

#pipe2: da decidere

#print matrix with the pipe1

#avviare dashboard

In [ ]:
#scrittura su file csv (per ora salto)
print(tweetTesting)

 apple       update  we   got her back  so     going to try and help her understand that she can go out  come back and be safe  


In [ ]:
#tokenization
tokens = nltk.word_tokenize(tweetTesting)
print(tokens)

['apple', 'update', 'we', 'got', 'her', 'back', 'so', 'going', 'to', 'try', 'and', 'help', 'her', 'understand', 'that', 'she', 'can', 'go', 'out', 'come', 'back', 'and', 'be', 'safe']


In [ ]:
#lemmatization
wn.ensure_loaded()
print(wn)

lemmatizer = WordNetLemmatizer()
lemmatized_sentence = []
lemmatized_sentence_joined = ""
tokensTagging = pos_tag(tokens)
print(tokensTagging)

<WordNetCorpusReader in '/home/lorenzo/nltk_data/corpora/wordnet'>
[('apple', 'NN'), ('update', 'NN'), ('we', 'PRP'), ('got', 'VBD'), ('her', 'PRP'), ('back', 'RB'), ('so', 'RB'), ('going', 'VBG'), ('to', 'TO'), ('try', 'VB'), ('and', 'CC'), ('help', 'VB'), ('her', 'PRP$'), ('understand', 'NN'), ('that', 'IN'), ('she', 'PRP'), ('can', 'MD'), ('go', 'VB'), ('out', 'RP'), ('come', 'VBN'), ('back', 'RB'), ('and', 'CC'), ('be', 'VB'), ('safe', 'JJ')]


In [ ]:

# pos tagging & lemmatization
for word, tag in tokensTagging:

    if tag.startswith('NN'):
        pos = 'n'
    elif tag.startswith('VB'):
        pos = 'v'
    else:
        pos = 'a'
    lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    
print(lemmatized_sentence)

['apple', 'update', 'we', 'get', 'her', 'back', 'so', 'go', 'to', 'try', 'and', 'help', 'her', 'understand', 'that', 'she', 'can', 'go', 'out', 'come', 'back', 'and', 'be', 'safe']


In [ ]:
lemmatized_sentence_set = ' '.join(lemmatized_sentence)
print(lemmatized_sentence_set)

apple update we get her back so go to try and help her understand that she can go out come back and be safe


In [ ]:
#removing stopwords
stopWords = stopwords.words('english')
filteredTokens = []
filteredTokens = [word for word in tokens if word not in stopWords]
print(filteredTokens)

['apple', 'update', 'got', 'back', 'going', 'try', 'help', 'understand', 'go', 'come', 'back', 'safe']


In [ ]:
#Processing
num_features = 1500

# Bag Of Words - uni-grams 
BOW_uni_svm_pipe = Pipeline([
        ('vect', CountVectorizer(ngram_range=(1, 1))),
        ('fselect', SelectKBest(chi2, k=num_features)),
        ('clf', svm.SVC()),
        ])

scoring = {'accuracy' : make_scorer(accuracy_score), 
            'precision' : make_scorer(precision_score,average='micro',labels=labels,zero_division=True),
            'recall' : make_scorer(recall_score,average='micro',labels=labels,zero_division=True), 
            'f1_score' : make_scorer(f1_score,average='micro',labels=labels,zero_division=True)}


NameError: name 'labels' is not defined

In [ ]:
# store results 
for pipe in pipes:
    results.append(cross_validate(estimator= pipe,
                    X=lemmatized_dataset,
                    y=labels,
                    cv=10,
                    scoring=scoring
                    )) 

In [ ]:
#append result
tmp_svm = {}
tmp_svm["Num Features"] = num_features
tmp_svm["BOW uni-grams Accuracy"] = np.mean(results[0]['test_accuracy'])
tmp_svm["BOW uni-grams Precision"] = np.mean(results[0]['test_precision'])
tmp_svm["BOW uni-grams Recall"] = np.mean(results[0]['test_recall'])
tmp_svm["BOW uni-grams F1 Score"] = np.mean(results[0]['test_f1_score'])